<a href="https://colab.research.google.com/github/albisbub/dighum/blob/master/notebooks/2020_07-28-Meeting-Week7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JATA Tools V1
> Data Fetchers for Interactivitys: AJHS Records, Finding Guides, and Loeb Images.

- toc:true
- branch: master
- badges: true
- comments: true
- author: Blaise
- permalink: /JATA1/
- categories: [fastpages, jupyter, dev, wip, check-in]




In [ ]:
#hide
!pip install requests
!pip install selenium

In [ ]:
#hide
!pip install beautifulsoup4

In [ ]:
#hide
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import bs4
import lxml.etree as xml
import urllib.request
import re
from pandas.io.html import read_html
from selenium import webdriver
from timeit import default_timer as timer
#hide
def striplist(list):
  out = []
  # print(list)
  for i in list:
    stin = str(i)
    split = (stin.split('>'))
    otherside = (split[1].split('<'))
    out_app = otherside[0]
    out.append(out_app)
    # b = str(i.split('>')[1])
    # print(b)
    # out.append(i)

  return out
def find_between( s, first, last ):
  try:
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        start = s.index( first ) + len( first )
        # end = s.index( last, start )
        return s[start:]
        # return "ERROR"
  except BaseException as e:
    print(e, first, last)
    return 'NA'


# Center For Jewish History Archives Scraper
 - Built in support for AJHS
 - Can be easily used for any repo on CJH's archive space
 - Could also be used in other archivespace scraping situations.


In [ ]:
#collapse-hide
class CJH_Archives:
  def scrape_all_records(object_type='records',start_page=1, stop_after_pages=0):
    if start_page <= 0:
      print("Must start at minimum of page 1")
      start_page=1
      page=start_page
    else:
      page = start_page 
    if object_type.upper() == 'RECORDS':
      print("Scraping All Individual Records")
      # page = start_page
      headless_url = "https://archives.cjh.org/repositories/3/objects?q%5B%5D=%2A&op%5B%5D=OR&field%5B%5D=keyword&from_year%5B%5D=&to_year%5B%5D=&limit=digital_object,archival_object&sort=title_sort%20asc&page="
      base_URL = str(headless_url + str(page))
    elif object_type.upper() == 'COLLECTIONS':
      # page = start_page
      print("Scraping Collections (Finding Aids)")
      headless_url = "https://archives.cjh.org/repositories/3/resources?q[]=%2A&op[]=&field[]=title&from_year[]=&to_year[]=&limit=resource&sort=title_sort%20asc&page="
      base_URL = str(headless_url + str(page))

    def scrape_record(name, link, web_page, object_type):
      # print(web_page, link)
      # (.+?)
      # meta_dict = find_between(str(i),'<script type="application/ld+json">','  </script>' )
      # meta_dict = re.findall(r'>(', str(web_page))
      title = (web_page.title)
      part_of = web_page.find_all('ul',{'class':'breadcrumb'})
      part_of = part_of[0].find_all('a')

      location_tupes = []
      for i in part_of:
        link = (str(i).split('"')[1])
        found_loc_name = (str(i).split('>')[1]).split('<')[0]
        tupp = (found_loc_name,link)
        location_tupes.append(tupp)
        # location_name = (str(i.split('>')[1])).split('<')[0]
      # stri = "<a href="
      # part_of = list(map(lambda st: str.replace(st,stri, ""), part_of)) 
      locs = (location_tupes)

      subnotes = web_page.find_all('div', {'class': 'upper-record-details'})[0].text
      # print(subnotes)
      div_data_1 = [("Name", name), ("Link",link)]


      acord = web_page.find_all('div', {'class': 'acc_holder clear'})[0].text
      
      acc_data = []

      if object_type.upper() == 'RECORDS':
        possible_fields_1=[
        "Scope and Contents",
        "Dates",
        "Language of Materials",
        "Access Restrictions",
        "Extent",
        ]
        possible_fields_2 = [
        "Related Names",
        "Digital Material",
        "Physical Storage Information",
        "Repository Details",

        ]
      elif object_type.upper() == 'COLLECTIONS':
        possible_fields_1=[
        "Scope and Content Note",
        "Dates",
        "Creator",
        "Access Restrictions",
        "Use Restrictions",
        "Conditions Governing Access",
        "Conditions Governing Use",
        "Extent",
        "Language of Materials"

        ]
        possible_fields_2 = [
        "Additional Description",
        "Subjects",
        "Related Names",
        "Finding Aid & Administrative Information",
        'Physical Storage Information',
        'Repository Details',

        ]


    
      ##subnotes
      b1 = []
      for i in possible_fields_1:
        if i in str(subnotes):
          out=True
        else:
          out = False
          missingTuple = (i, '')
          div_data_1.append(missingTuple)      
        b1.append(out)
      ##accordian

      b2=[]

      for i in possible_fields_2:
        if i in str(acord):
          out=True
        else:
          out = False
          missingTuple = (i, '')
          div_data_1.append(missingTuple)
        b2.append(out)

      # print(b1, b2)
      xs=possible_fields_1
      ys=b1
      # sec_1_heads = [x for x, y in zip(xs, ys) if y == 'True']
      filtered1 = np.array(xs)[np.array(ys)]
      xs=possible_fields_2
      ys=b2  
      filtered2 = np.array(xs)[np.array(ys)]

      # sec_2_heads =  [x for x, y in zip(xs, ys) if y == 'True']
      # print(filtered1,filtered2,'xyz')
      indexer = 0
      for i in filtered1:
        # print(len(filtered1),len(filtered2), (indexer))
        first = i

        try:
          next = filtered1[indexer+1]
        except BaseException as e:
          next = '$$$'
        # print(first, next)
        value = find_between(subnotes, first, next)
        # print(first, next, value)
        value = value.replace('\n',' ').strip().replace('\t', ' ')
        # print(first, next, value)
        val = (i,value)
        div_data_1.append(val)
        indexer+=1
        # print(indexer, first, next)
      indexer = 0
      for i in filtered2:
        first = i
        try:
          next = filtered1[indexer+1]
        except BaseException as e:
          next = '$$$'
          # print(first,next)
        value = find_between(acord, first, next)
        # print(first, next, value)
        value = value.replace('\n',' ').strip().replace('\t', ' ')
        val = (i,value)
        div_data_1.append(val)
        indexer+=1
        # print(indexer, first, next)
      
      # exit
      
      bigList = (div_data_1)
      return tuple(bigList)
    URL = base_URL
    web_page = BeautifulSoup(requests.get(URL, {}).text, "lxml")
    pagnation = web_page.find_all('ul',{'class':'pagination'})[0].find_all('li')
    next_link = (web_page.find_all('li',{'class':'next'})[0]).find('a',href=True)
    linkky = str(next_link)
    nextPage_ = str("https://archives.cjh.org" + (linkky.split('"')[1]))
    # exit
    # print(pagnation)
    pageList = []
    s_pages = []
    for i in pagnation:
      number = str(i).split('>')[2].split('<')[0]
      pageList.append((number))
    # print("Pages", pageList)
    # break
    test_list=[]
    for i in pageList:
      try:
        # print(i)
        # print( int(i))
        test_list.append(int(i))
      except:
        pass
    # test_list = [int(i) for i in pageList if not (i).isdigit()]
    # print(test_list)
    last_page__ = (max(test_list))
    __lastPage = last_page__ - (last_page__ - stop_after_pages)
    print()
    # exit
    page_counter = 1
    while page_counter < __lastPage:
      row_list = []
      pagez= page_counter
      print("Scraping Page", page_counter)

      page_current = page_counter
      URL = str(headless_url + str(page_current))
      web_page = BeautifulSoup(requests.get(URL, {}).text, "lxml")
      h3s = web_page.find_all('h3')
      # summs = web_page
      tupleList = []
      for i in h3s:
        # print(i)
        try:
          link = ((str(i).split('href="')[1]).split('"'))[0]
          name = (str(i).split('">'))[1].split("</a")[0]
          # print(link, name)
          # break
          data_tuple = (name ,str("https://archives.cjh.org" + link), link)
          tupleList.append(data_tuple)
          
        except BaseException as e:
          print(e, i)
      page_counter+=1
    archIndex = pd.DataFrame.from_records(tupleList, columns = ['Names', 'Link', 'Location'])

    # ...
    counter = 0
    for i in archIndex.itertuples():


      counter +=1

      name = i.Names
      link = i.Link
      link123 = link
      Location=i.Location
      web_page = BeautifulSoup(requests.get(link, {}).text, "lxml")

      record_row = scrape_record(name, link123, web_page,object_type.upper() )
      row_list.extend(record_row)
      print("Record: ",counter, link123)


      s_pages.extend(row_list)

    
    d = {}
    for x, y in s_pages:
        d.setdefault(x, []).append(y)   
    df = pd.DataFrame.from_records(d).drop_duplicates()
    if object_type.upper() == 'RECORDS':

      df[['Date_1','Date_2']] = (df['Dates'].str.split('–', n=1,expand=True))
    else:
      
      df['Use Terms'] = df['Use Restrictions']+df['Conditions Governing Use']
      # df1.replace('NA',np.nan,inplace=True)
      df['Access Terms'] = df[ 'Access Restrictions']+df['Conditions Governing Access']
      dropThese = [
              'Use Restrictions',
              'Conditions Governing Use',
              'Access Restrictions',
              'Conditions Governing Access',
              ]
      df.drop(columns=dropThese,inplace=True)

      # df1 = df1.apply(lambda x: None if x.isnull().all() else ';'.join(x.dropna()), axis=1)
      
    return (df)

  def __init__(self, repo):
    self.repo = repo

  def get_meta_data(self, object_type,page_to_start_at,maximum_pages_to_scrape):

    if self.repo.upper() == 'AJHS':
      print('Creating CJHA Scraper Object for AJHS')
      self.meta_df = scrape_all_records(object_type,page_to_start_at,maximum_pages_to_scrape)
      return self.meta_df
    else:
      print("WIP WIP WIP WIP WIP WIP")
      pass

## Building AJHS Archive Datasets

The below line of code can be used to scrape the archive for a given number of pages (input 0 for all records). There are two object types, records and collections. Collections are digitized finding aids and records are all contained in some sort of collection. Some are under multiple collections. The below lines of code generate dataframes for the first 3 pages of records and collections

In [ ]:
#collapse-hide
# %%capture
#records
ajhs_recs = CJH_Archives('ajhs').get_meta_data('records', 1, 3)
#collections
ajhs_cols= CJH_Archives('ajhs').get_meta_data('collections', 1, 3)

## Output For Records

In [ ]:
#hide-input
ajhs_recs

,Access Restrictions,Dates,Digital Material,Extent,Language of Materials,Link,Name,Physical Storage Information,Related Names,Repository Details,Scope and Contents,Date_1,Date_2
0,This collection is open to all researchers by ...,1991–1994,Request from an off-site storage location ...,From the Collection: 902 Linear Feet (748 B...,From the Collection: The collection i...,/repositories/3/archival_objects/272963,"056 Flint, MI—Noreen Kasle, 1991–1994","Box: 296, Folder: 38 (Mixed Materials) ...",Creator From the Collection: HIAS (Agency) ...,Repository Details Part of the American Jewis...,Carolyn Agress file,1991,1994
2,This collection is open to all researchers by ...,1992–1993,Request from an off-site storage location ...,From the Collection: 902 Linear Feet (748 B...,From the Collection: The collection i...,/repositories/3/archival_objects/272963,"057 Ft. Lauderdale, FL—Barbara Wiener, 1992–...","Box: 296, Folder: 39 (Mixed Materials) ...",Creator From the Collection: HIAS (Agency) ...,Repository Details Part of the American Jewis...,From the Sub-Series: Campaigns we...,1992,1993
5,This collection is open to all researchers by ...,1992–1995,Request from an off-site storage location ...,From the Collection: 902 Linear Feet (748 B...,From the Collection: The collection i...,/repositories/3/archival_objects/272963,"058 Washington, D.C.—Jerome Dick, 1992–1995","Box: 296, Folder: 42 (Mixed Materials) ...",Creator From the Collection: HIAS (Agency) ...,Repository Details Part of the American Jewis...,Carolyn Agress file,1992,1995
9,This collection is open to all researchers by ...,1994–1995,Request from an off-site storage location ...,From the Collection: 902 Linear Feet (748 B...,From the Collection: The collection i...,/repositories/3/archival_objects/272963,"058 Washington, D.C.—Jerome Dick, 1994–1995","Box: 296, Folder: 43 (Mixed Materials) ...",Creator From the Collection: HIAS (Agency) ...,Repository Details Part of the American Jewis...,Carolyn Agress file,1994,1995
14,This collection is open to all researchers by ...,1996,Request from an off-site storage location ...,From the Collection: 902 Linear Feet (748 B...,From the Collection: The collection i...,/repositories/3/archival_objects/272963,"058 Washington, D.C.—Jerome Dick, 1996","Box: 296, Folder: 44 (Mixed Materials) ...",Creator From the Collection: HIAS (Agency) ...,Repository Details Part of the American Jewis...,Carolyn Agress file,1996,None
20,This collection is open to all researchers by ...,1997,Request from an off-site storage location ...,From the Collection: 902 Linear Feet (748 B...,From the Collection: The collection i...,/repositories/3/archival_objects/272963,"058 Washington, D.C.—Jerome Dick, 1997","Box: 296, Folder: 45 (Mixed Materials) ...",Creator From the Collection: HIAS (Agency) ...,Repository Details Part of the American Jewis...,Carolyn Agress file,1997,None
27,This collection is open to all researchers by ...,1983–1993,Request from an off-site storage location ...,From the Collection: 902 Linear Feet (748 B...,From the Collection: The collection i...,/repositories/3/archival_objects/272963,"058 Washington, D.C.—Jerome Dick—1 of 2, 198...","Box: 296, Folder: 40 (Mixed Materials) ...",Creator From the Collection: HIAS (Agency) ...,Repository Details Part of the American Jewis...,From the Sub-Series: Campaigns we...,1983,1993
35,This collection is open to all researchers by ...,1983–1993,Request from an off-site storage location ...,From the Collection: 902 Linear Feet (748 B...,From the Collection: The collection i...,/repositories/3/archival_objects/272963,"058 Washington, D.C.—Jerome Dick—2 of 2, 198...","Box: 296, Folder: 41 (Mixed Materials) ...",Creator From the Collection: HIAS (Agency) ...,Repository Details Part of the American Jewis...,From the Sub-Series: Campaigns we...,1983,1993
44,This collection is open to all researchers by ...,1994–1995,Request from an off-site storage location ...,From the Collection: 902 Linear Feet (748 B...,F

## Output for Collections

In [ ]:
#hide-input
ajhs_cols

,Additional Description,Creator,Dates,Extent,Finding Aid & Administrative Information,Language of Materials,Link,Name,Physical Storage Information,Related Names,Repository Details,Scope and Content Note,Subjects,Use Terms,Access Terms
0,Overview This collection is comprise...,,1908-1973 Language of Materials ...,75 Manuscript Boxes (37.75 feet and one oversi...,Title Guide to the Papers of Ad...,"The collection is in English, German, French, ...",/repositories/3,Admiral Lewis Lichtenstein Strauss Papers,,,Repository Details Part of the American Jewis...,The Papers of Admiral Lewis L. Strauss reflect...,are arranged according to series. Folders are ...,There may be some restrictions on the use of t...,The collection is open to all researchers by p...
2,Overview Printer's proof of Kraus' e...,"Kraus, Adolf, 1850-1928 (Person)",ca. 1911,.25 Linear Feet,Title Guide to the Adolf Kraus ...,,/repositories/3,Adolf Kraus biographical data,,,Repository Details Part of the American Jewis...,Printer's proof of Kraus' entry in Who's Who i...,,,
5,Overview This collection consists of...,"Lorch, Adolf, 1883-1971 (Person) ...",1896-2013 Majority of ...,"0.75 Linear Feet (1 manuscript box, 1 half-man...",Title Guide to the Adolf Lorch ...,The collection is in English and German. ...,/repositories/3,Adolf Lorch (1883-1971) Papers,,"Creator Lorch, Adolf, 1883-1971 (Per...",Repository Details Part of the American Jewis...,This collection consists of materials related ...,Correspondence Hudson (N....,Information concerning the literary rights may...,[If you have any exceptions to the AJHS-approv...
9,Overview Contains three checks signe...,"Sutro, Adolph, 1830-1898 (Person)","undated, 1896-1897",.25 Linear Feet,Title Guide to the Adolph Heinr...,English,/repositories/3,Adolph Heinrich Joseph Sutro papers,"Container: Consolidated Box P9, Folder: P-318 ...","Creator Sutro, Adolph, 1830-1898 (Pe...",Repository Details Part of the American Jewis...,"Contains three checks signed by Sutro; , and a...",Finance Public health ...,"No permission is required to quote, reproduce ...","The collection is open to all researchers, exc..."
14,Overview Consists of a collection of...,"Sabath, Adolph Joachim, 1866-1952 (Per...",1909-1952,0.5 Linear Feet,Title Guide to the Adolph J. Sa...,eng Access Restrictions ...,/repositories/3,Adolph J. Sabath papers,,"Creator Sabath, Adolph Joachim, 1866-1952 ...",Repository Details Part of the American Jewis...,Consists of a collection of bills and resoluti...,Emigration and immigration law ...,,None Biographical Note ...
20,Overview Included in this collection...,"Solomons, Adolphus S. (Adolphus Simeon), 1826-...","undated, 1841-1966","3.7 Linear Feet (4 manuscript boxes, 1 oversiz...",Title Guide to the Papers of Ad...,The collection is in English. Access...,/repositories/3,"Adolphus Simeon Solomons, papers",,"Creator Solomons, Adolphus S. (Adolphus Sime...",Repository Details Part of the American Jewis...,Included in this collection are papers which r...,American Red Cross Americ...,"No permission is required to quote, reproduce ...","The collection is open to all researchers, exc..."
27,Overview Collection consists of annu...,"Aguilar Free Library (New York, N.Y.) ...",1899-1903,1 Folders,Title Guide to the Aguilar Free...,eng Extent 1 Folders,/repositories/3,Aguilar Free Library Society records,,"Creator Aguilar Free Library (New York, N.Y....",Repository Details Part of the American Jewis...,Collection consists of annual reports from 189...,"Educational Alliance (New York, N.Y.) ...",,
35,Overview The collection contains pho...,"Cohen, Alan, L. (Person) Lang...","1989, 1993",1 Manuscript Boxes (1/2 linear foot),Title Guide to the Papers of Al...,The collection is in English. Access...,/repositories/3,Alan L. Cohen Papers,,"Creator Cohen, Alan, L. (Person) ...",Repository Details Part of the American Jewis...,The collection contains 203 photographs taken ...,Antisemitism Emigration a...,Information concerning the literary 

# Loeb Data Scraper

The [Loeb data scraper](https://loebjewishportraits.com)fetches meta deta and can download images for paintings, silhouettes, and photographs from the archive (or all of the above).

In [ ]:
#collapse-hide
class loeb:
  """
  This class can be used to interact with the loeb image data base.
  The init funciton takes 1 argument which is the type of data to retreive.
  The input should be one of the following : 'paintings', silhouettes, photographs, or 'all'
  """
  def __init__(self, data_set='paintings'):

    def scrape_loeb(URL):
      requests.get(URL)
      web_page = bs4.BeautifulSoup(requests.get(URL, {}).text, "lxml")
      table = web_page.find_all('portfolio')
      div = web_page.find(id="portfolio")
      linkList = web_page.find_all('div',{'class':'work-info'})
      df_dict = []
      for links in linkList:
        twolinks = links.find_all('a', href=True)
        details = str(twolinks[0]).split('"')[1]
        img = str(twolinks[1]).split('"')[3]
        new_df_tuple = {'info_link':details, 'img_link':img}
        df_dict.append(new_df_tuple)
      
      listOfDfs = []
      counter = 0
      df = pd.DataFrame.from_records(df_dict)
      for i in df.itertuples():
        img = i.img_link
        info = i.info_link
        # print(info)
        
        # print(info)
        # download_image(img,'test.jpg')
        
        profile = bs4.BeautifulSoup(requests.get(info, {}).text, "lxml")
        img = str(profile.find_all('img',src=True)[0]).split('"')[3]
        # print(img)
        # print(profile)
        # print(profile)
        a = profile.find_all('h4')
        # print(a)
        b = profile.find_all("h3")

        # bio = profile
        linkts = str(profile.find_all('a',{'id':'viewzoom'},href=True)[1]).split('"')[1]

        def scrape_bio_loeb(url):
          bio = bs4.BeautifulSoup(requests.get(url, {}).text, "lxml")
          abc=str(bio.find_all('p')[1]).replace("<p>", " ")
          abcd=(str(abc).replace('</p>', " "))
          bio_text = str(str(abcd.replace('<i>',' ')).replace("</i>",' '))
          s = bio_text
          bio_plain = re.sub(r'<.+?>', '', s)
          if 'Lorem ipsum dolor sit amet,' in bio_plain:
            bio_plain = ''
          if "Lorem ipsum dolor sit amet," in s:
            s = ''
          # bio_escapes = re.sub(r'>.+?<', '', s)
          return bio_plain, s
        
        bio__ = scrape_bio_loeb(linkts)
        # print(bio__)
        

        # print(linkts,len(linkts), "hkgdfsjhsfgakljhashlf")
        # break
        headers4 = striplist((a))
        headers4_ = ['Name']
        for i in headers4:
          headers4_.append(i)
        # headers4_ = .extend(headers4)
        headers3 = striplist( b)
        # print(headers4_, headers3)
        # break
        headers4_ = headers4_[:-1]
        headers4_.append('Bio_Plain')
        headers3.append(bio__[0])

        headers4_.append('Bio_Links')
        headers3.append(bio__[1])
        df1 = pd.DataFrame({'Label':headers4_ , 'Value': headers3})

        # name_for_file = headers[0][1]
        # print(name_for_file, headers, headers[0])
        self.image_cache.append((img, df1))


        listOfDfs.append(df1)
        # download_image(img, str(str(counter) + '.jpg'))
        counter+=1


        self.list_of_dfs.extend(listOfDfs) 

    self.list_of_dfs = []
    self.image_cache = []
    
    if data_set.upper() == 'ALL':
      data_options = ['paintings', 'silhouettes', 'photographs']

      for i in data_options:
        print(i)
        URL = str("http://loebjewishportraits.com/" + i + '/')
        scrape_loeb(URL)

    else:
      try:
          
        URL = str("http://loebjewishportraits.com/" + data_set + '/')
        scrape_loeb(URL)
      except BaseException as e:
        print(e)
        print("Could not find a data set for: ", data_set, "Make sure you input either 'paintings', 'silhouettes', or 'photographs'!")


  def get_meta_data(self, export=False):
    """
    returns a meta dataframe with each painting as an entry in a row

    export can be csv or excel
    """
    listy = self.list_of_dfs
    transposed = [thing.transpose() for thing in listy]

    cc = 1
    newList = []
    for i in transposed:
      # print(len(i.columns))
      new_cols = (i.iloc[0])
      i.columns = new_cols
      i.drop(i.index[0], inplace= True)

      
    long_df_of_entrys = pd.concat(transposed)
    long_df_of_entrys.set_index('Name')

    return long_df_of_entrys.reset_index()


  def download_images(self):
    def download_image(link,filename):
      urllib.request.urlretrieve(link, filename)
      # print('image saved to temp directory')



    for i in self.image_cache:
      name = (i[1].Value.iloc[0])
      
      fileName = str(name + '.jpg')

      try:

        download_image(i[0],fileName)
        print('Saved', fileName, 'to current directory')

      except BaseException as e:
        print("Could not download:", fileName, "Error:",e)


## Scraping Meta Data and Download Locations for Selected Image Type

In [ ]:
paintings = loeb()


## Building a MetaData Dataset for the Paintings

In [ ]:
meta_data = paintings.get_meta_data()


## Output For Painting MetaData

In [ ]:
#hide-input
meta_data

,index,Name,Life Span,Artist,Painting Date,Medium,Repository,Bio_Plain,Bio_Links,Dimensions,Artist Life Span
0,Value,Richea Myers Cohen Gratz,1731-1801,Artist unknown,ca. 1750,Oil on canvas,Mikveh Israel,"In 1729 Samuel Myers-Cohen, a militia veteran...","In 1729 <a class=""intlink"" href=""http://loebj...",NaN,NaN
1,Value,Richea Myers Cohen Gratz,1731-1801,Artist unknown,ca. 1750,Oil on canvas,Mikveh Israel,"In 1729 Samuel Myers-Cohen, a militia veteran...","In 1729 <a class=""intlink"" href=""http://loebj...",NaN,NaN
2,Value,Sarah Hamburger,NaN,Artist unknown,ca. 1850,Oil on canvas,Jewish Museum of Maryland,,,16 1/2 x 13 1/2 in.,NaN
3,Value,Richea Myers Cohen Gratz,1731-1801,Artist unknown,ca. 1750,Oil on canvas,Mikveh Israel,"In 1729 Samuel Myers-Cohen, a militia veteran...","In 1729 <a class=""intlink"" href=""http://loebj...",NaN,NaN
4,Value,Sarah Hamburger,NaN,Artist unknown,ca. 1850,Oil on canvas,Jewish Museum of Maryland,,,16 1/2 x 13 1/2 in.,NaN
...,...,...,...,...,...,...,...,...,...,...,...
53296,Value,Miriam Simon Gratz,1749-1808,Jane Cooper Sully Darley,NaN,Oil on canvas,Rosenbach Museum and Library,Miriam Simon was the oldest surviving child b...,Miriam Simon was the oldest surviving child b...,NaN,1807-1877
53297,Value,Rebecca Gratz,1781-1869,Thomas Sully,1831,Oil on board,Rosenbach Museum and Library,"Rebecca Gratz, the seventh of twelve children...","<a class=""intlink"" href=""http://loebjewishpor...",20 x 17 in.,1783-1872
53298,Value,Rebecca Gratz,1781-1869,Thomas Sully,1830,"Oil on canvas, mounted on masonite",Delaware Art Museum,"Rebecca Gratz, the seventh of twelve children...","<a class=""intlink"" href=""http://loebjewishpor...",20 x 17 in.,1783-1872
53299,Value,Rebecca Gratz,1781-1869,Thomas Sully,1858,Oil on canvas,American Jewish Historical Society,"Rebecca Gratz, the seventh of twelve children...","<a class=""intlink"" href=""http://loebjewishpor...",20 x 16 in.,1783-1872


## Batch Downloading Paintings (Takes a while!)

In [ ]:
# paintings.download_images()